In [1]:
import nltk
from collections import Counter
import numpy as np

In [2]:
def find_token(deterministic, counter):
    if deterministic:
        max_value = max(counter.values())
        candidates = sorted([key for key in counter if counter[key] == max_value])
        return candidates[0]
    else:
        counts = np.array(list(counter.values()))
        p = counts/sum(counts)
        return np.random.choice(np.array(list(counter.keys())), size = 1, p = p)[0]

In [15]:
# Assume n <= length of sentence
def create_token(sentence, backoff, deterministic, n):
    if n > 1:
        initial_prefix = sentence[-(n-1):]
    else:
        initial_prefix = []
    found_token = False
    while not found_token:
        length = len(initial_prefix)
        if length > 1:
            if tuple(initial_prefix) in backoff[length]:
                found_token = True
                # return token either deterministically or not
                return find_token(deterministic, backoff[length][tuple(initial_prefix)])
            initial_prefix = initial_prefix[1:]
        if length == 1:
            if initial_prefix[0] in backoff[length]:
                found_token = True
                # return token either deterministically or not
                return find_token(deterministic, backoff[length][initial_prefix[0]])
            initial_prefix = []
        else:
            # return token from backoff[0]
            return find_token(deterministic, backoff[0])

In [4]:
def finish_sentence(sentence, n, corpus, deterministic=False):
    # Create backoff list
    backoff = []
    backoff.append(Counter(corpus))
    # Handle 2-gram case
    dic = {}
    for i in range(0,len(corpus)-1):
        prefix = corpus[i]
        next_word = corpus[i+1]
        if prefix not in dic:
            dic[prefix] = Counter()
        if next_word not in dic[prefix]:
            dic[prefix][next_word] = 0
        dic[prefix][next_word] += 1
    backoff.append(dic)
    # Handle 3-grams to n-grams
    for j in range(2, n):
        dic = {}
        for i in range(0,len(corpus)-j):
            prefix = tuple(corpus[i:i+j])
            next_word = corpus[i+j]
            if prefix not in dic:
                dic[prefix] = Counter()
            if next_word not in dic[prefix]:
                dic[prefix][next_word] = 0
            dic[prefix][next_word] += 1
        backoff.append(dic)
    
    while True:
        token = create_token(sentence, backoff, deterministic, n)
        sentence.append(token)
        if token == "." or token == "!" or token == "?" or len(sentence) == 15:
            return sentence

In [5]:
sentence = ["she", "was", "not"]
n = 3
corpus = [
w.lower() for w in
nltk.corpus.gutenberg.words("austen-sense.txt")
]
deterministic = True

In [ ]:
print(finish_sentence(sentence, n, corpus, False))

In [16]:
sentence = ["she", "was", "not"]
print(finish_sentence(sentence, 2, corpus, True))

['she', 'was', 'not', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',']


In [14]:
len([])

0